In [1]:
from pathlib import Path

In [2]:
import cv2

In [ ]:
from skimage.transform import resize

In [ ]:
from skimage.io import imread, imsave

In [3]:
FORCE = False

In [4]:
USE_CV = True

def load_img(path):
    if USE_CV:
        return cv2.imread(path)
    else:
        return imread(path)

def save_img(path, img):
    if USE_CV:
        return cv2.imwrite(path, img)
    else:
        return imsave(path, img)
    
def resize_img(img, size):
    if USE_CV:
        return cv2.resize(img, size)
    else:
        return resize(
            img,
            size,
            mode="reflect"
            #anti_aliasing=True
        )

In [5]:
if USE_CV:
    size = (640, 192)
else:
    size = (192, 640)

In [7]:
roots = Path('.').iterdir()
roots=list(roots)
roots = [root for root in roots if root.is_dir() and '2011_' in root.name]
roots

[PosixPath('2011_09_29'),
 PosixPath('2011_09_30'),
 PosixPath('2011_09_26'),
 PosixPath('2011_09_28'),
 PosixPath('2011_10_03')]

size = (188, 620)
#size = (620, 188)
for root in roots:
    folders = root.glob("*_sync")
    for folder in folders:
        img02r = folder /'image_02_{}_{}'.format(*size)/'data'
        img03r = folder /'image_03_{}_{}'.format(*size)/'data'

        for which in (img02r, img03r):
            !rm -r {str(which)}

In [8]:
from tqdm import tqdm_notebook as tqdm

In [9]:
pbar=tqdm(total=1)
pbar.update(1)
pbar.close()

In [ ]:
timgs0, timgs1, timgs2, paras = [], [], [], []


readimgs = []
saveimgs = []

for root in roots:
    folders = root.glob("*_sync")
    for folder in folders:
        img02 = folder /'image_02'/'data'
        img03 = folder /'image_03'/'data'
        img02r = folder /'image_02_{}_{}'.format(*size)/'data'
        img03r = folder /'image_03_{}_{}'.format(*size)/'data'
        
        img02r.mkdir(parents=True, exist_ok=True)
        img03r.mkdir(parents=True, exist_ok=True)
        for which, which_r in ((img02, img02r), (img03, img03r)):
            imgs = sorted([img for img in which.iterdir()])
            readimgs += imgs
            saveimgs += [which_r/img.name for img in imgs]

total = len(readimgs); total                

In [ ]:
from multiprocessing import Process, Lock

def f(readimgs, saveimgs):
    pbar = tqdm(total=len(readimgs))
    for img, saveimg in zip(readimgs, saveimgs):
        if not saveimg.exists() or FORCE:
            resized = resize_img( load_img(str(img)), size)
            save_img(str(saveimg), resized)        
        pbar.update(1)
    pbar.close

In [ ]:
ncores = 6
space = int( total / ncores ) + 1 
ps = []
space

In [ ]:

for num in range(ncores):
    p = Process(target=f,
                args=(readimgs[space*num: space*(num+1)],
                      saveimgs[space*num: space*(num+1)],
                      )
               )
    ps.append(p)
    
for p in ps: p.start()

In [ ]:
for p in ps: p.close()


In [ ]:
pbar = tqdm(total=total)
for root in roots:
    folders = root.glob("*_sync")
    for folder in folders:
        img02 = folder /'image_02'/'data'
        img03 = folder /'image_03'/'data'
        img02r = folder /'image_02_{}_{}'.format(*size)/'data'
        img03r = folder /'image_03_{}_{}'.format(*size)/'data'
        
        img02r.mkdir(parents=True, exist_ok=True)
        img03r.mkdir(parents=True, exist_ok=True)
        for which, which_r in ((img02, img02r), (img03, img03r)):
            imgs = sorted([img for img in which.iterdir()])
            
            for img in imgs:
                img_r = which_r/img.name
                if not img_r.exists() or FORCE:
                    resized = resize_img( load_img(str(img)), size)
                    save_img(str(img_r), resized)
                pbar.update(1)
pbar.close()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from collections import namedtuple

In [ ]:
CAMPARA = namedtuple('campara',['fx','fy','cx','cy'])
def get_campara(date):
    cam = date/'calib_cam_to_cam.txt'
    camret = {}
    with cam.open('r') as f:
        for l in f:
            kv = l.rstrip().split(':')
            k, v = kv[0], kv[1]
            v= v.split(' ')[1:]
            camret[k] = v
    cam2para = np.array(tuple(float(e) for e in camret['P_rect_02'])).reshape(3,4)
    return CAMPARA(fx = cam2para[0,0], fy = cam2para[1,1], cx = cam2para[0,2], cy = cam2para[1,2])

In [ ]:
def gene_tri(imgs):
    return imgs[0:-2], imgs[1:-1], imgs[2:]

In [ ]:
timgs0, timgs1, timgs2, paras = [], [], [], []
for root in roots:
    para = get_campara(root)
    folders = root.glob("*_sync")
    for folder in folders:
        img02r = folder /'image_02_{}_{}'.format(*size)/'data'
        img03r = folder /'image_03_{}_{}'.format(*size)/'data'
        
        for which in (img02r, img03r):
            imgs = sorted([str(img) for img in which.iterdir()])
            imgs0, imgs1, imgs2 = gene_tri(imgs)
            timgs0 += imgs0
            timgs1 += imgs1
            timgs2 += imgs2
            paras += [para]*len(imgs0)
paras = tuple(zip(*paras))

In [ ]:
train = pd.DataFrame(dict(t0=timgs0, t1=timgs1,t2=timgs2,
                          fx=paras[0],fy=paras[1],
                          cx=paras[2],cy=paras[3])
                    )

train[['fx','fy','cx','cy']]=train[['fx','fy','cx','cy']]/2

In [ ]:
train.head()

val = train.iloc[int(len(train)*0.9):]
train = train.iloc[:int(len(train)*0.9)]

if USE_CV:
    val.to_csv('validation_{1}_{0}.csv'.format(*size), index=False)
    train.to_csv('training_{1}_{0}.csv'.format(*size), index=False);
else:
    val.to_csv('validation_{}_{}.csv'.format(*size), index=False)
    train.to_csv('training_{}_{}.csv'.format(*size), index=False);

In [ ]:
train.head()

In [ ]:
val.head()